In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_selection import mutual_info_regression
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

features = ["HomePlanet", "CryoSleep","Cabin","Destination","Age","VIP","RoomService","FoodCourt","ShoppingMall","Spa","VRDeck","Group"]
y_data = data.pop("Transported").astype("float")
data["Group"] = data["PassengerId"].apply(lambda x: int(x[-2:])).astype("float64")
x_data = data[features]

test["Group"] = test["PassengerId"].apply(lambda x: int(x[-2:])).astype("float64")
x_test = test[features]

In [3]:
x_data[["Deck","Num","Side"]] = x_data.pop("Cabin").str.split("/",expand=True)
x_data["Num"] = x_data["Num"].astype("float64")
x_data["VIP"] = x_data["VIP"].astype("float64")
x_data["CryoSleep"] = x_data["CryoSleep"].astype("float64")
x_test[["Deck","Num","Side"]] = x_test.pop("Cabin").str.split("/",expand=True)
x_test["Num"] = x_test["Num"].astype("float64")
x_test["VIP"] = x_test["VIP"].astype("float64")
x_test["CryoSleep"] = x_test["CryoSleep"].astype("float64")

/tmp/ipykernel_19/1884274725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_data[["Deck","Num","Side"]] = x_data.pop("Cabin").str.split("/",expand=True)
/tmp/ipykernel_19/1884274725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_data[["Deck","Num","Side"]] = x_data.pop("Cabin").str.split("/",expand=True)
/tmp/ipykernel_19/1884274725.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [4]:
x_data["Side"].value_counts()
x_data.tail()
#print(x_data.info())
#print(x_test.info())

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Deck,Num,Side
8688,Europa,0.0,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,1.0,A,98.0,P
8689,Earth,1.0,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,G,1499.0,S
8690,Earth,0.0,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,1.0,G,1500.0,S
8691,Europa,0.0,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,1.0,E,608.0,S
8692,Europa,0.0,TRAPPIST-1e,44.0,0.0,126.0,4688.0,0.0,0.0,12.0,2.0,E,608.0,S


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

categorical = ["HomePlanet", "Destination", "Deck", "Side"]
numerical = [i for i in x_data.columns if i not in categorical]

cat = Pipeline(steps=[
    ("imp", SimpleImputer(strategy="constant")),
    ("enc", OneHotEncoder())
])
prep = ColumnTransformer(transformers=[
    ("num", SimpleImputer(), numerical),
    ("cat", cat, categorical)
])


In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_validate

model = XGBClassifier(n_estimators=10,learning_rate=0.1)
final = Pipeline(steps=[
    ("prep",prep),
    ("model", model)
])

cross_validate(final, x_data, y_data)
final.fit(x_data,y_data)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['CryoSleep', 'Age', 'VIP',
                                                   'RoomService', 'FoodCourt',
                                                   'ShoppingMall', 'Spa',
                                                   'VRDeck', 'Group', 'Num']),
                                                 ('cat',
                                                  Pipeline(steps=[('imp',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('enc',
                                                                   OneHotEncoder())]),
                                                  ['HomePlanet', 'Destination',
                                                   'Deck', 'Side'])])),
                ('model',
                 XGBClassifier(base_score=N...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=10, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [7]:
preds = final.predict(x_test)
out = pd.DataFrame()
out["PassengerId"] = test["PassengerId"]
out["Transported"] = preds
out.to_csv("submission.csv", index=False)